In [36]:
import pandas as pd
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer,PorterStemmer
lemmatizer=WordNetLemmatizer()
stem=PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [37]:
df=pd.read_csv('twitter_validation.csv',header=None)
df.columns=['id','company','sentiment','raw-tweet']
df=df[df['sentiment']!='Irrelevant']
df=df[df['sentiment']!='Neutral']
tweets=df['raw-tweet'].values

data pre-processing


In [38]:
processed_tweets=[]
for tweet in tweets:
  tweet = (re.sub(r'[^\w\s]','',tweet)).lower()
  tweet = [word for word in tweet.split(' ') if word not in stopwords.words('english') and len(word)>0]

  tweet = [lemmatizer.lemmatize(word) for word in tweet]
  processed_tweets.append(' '.join(tweet))

In [39]:
len(processed_tweets)

543

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer()
X=vec.fit_transform(processed_tweets)

df2=pd.DataFrame(X.toarray(),columns=vec.get_feature_names_out())
df2.head(2)

,05,10,100,1000,1000000,100lp,11,1131am,117,118,...,youtubexjo21nvywqa,youtubeztl5ajdb48,yuumi,zakawaqar,zen,zero,zone,zoom,zoom_us,𝐍𝐄𝐖𝐒𝐔𝐏𝐃𝐀𝐓𝐄𝐒
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
import joblib


In [42]:
model = joblib.dump(X,'Tf_idf_model.joblib')
df2.to_csv('tf_idf_enc.csv',index=False)

# Preprocess the data for model building

In [43]:
x=X.toarray()
y=pd.get_dummies(df['sentiment']).values[:,1:]

In [46]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

Model Training

In [54]:
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()
model.fit(x_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MultinomialNB()

In [55]:
y_pred = model.predict(x_test)

In [56]:
import numpy as np

In [57]:
model.score(x_test,y_test)

0.7614678899082569

In [58]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7614678899082569